In [1]:
import tensorflow as tf
import numpy as np
import math
import pandas as pd

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import Reshape, MaxPooling2D
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import load_model
from keras.models import model_from_json
from tensorflow import keras

Using TensorFlow backend.


In [2]:
def xyPart(data):
    Xdata = data.iloc[:,0:177]
    Ydata = []
    for elem in data.iloc[:,177]:
        if elem == "Low":
            Ydata.append(0)
        elif elem == "High":
            Ydata.append(1)
    Ydata = np.array(Ydata)
    return Xdata, Ydata

In [3]:
# Load and preprocess data
data = pd.read_csv('HLMdesc.txt', sep="\t")
data = data.drop(columns = ['Protocol Number', 'Experiment Start Time', 'Experiment Stop Time', 'Result Type Name', 'Result Operator', 'Result Number Rounded',\
 'Project NumName', 'Smiles', 'Leonumber', 'DResult Text', 'DResult Text_1'])
data = data.drop(columns = ['Protocol Number_1', 'Experiment Start Time_1', 'Experiment Stop Time_1', 'Result Type Name_1', 'Result Operator_1', 'Result Number Rounded_1', 'Project NumName_1',\
       'Leonumber_1', 'origSmiles_1', 'ClappClass_1'])
print(data.columns)


Index(['rdk.fr_C_O_noCOO', 'rdk.Chi4v', 'rdk.fr_Ar_COO', 'rdk.fr_SH',
       'rdk.Chi4n', 'rdk.SMR_VSA10', 'rdk.fr_para_hydroxylation',
       'rdk.fr_barbitur', 'rdk.fr_halogen', 'rdk.fr_dihydropyridine',
       ...
       'rdk.PEOE_VSA9', 'rdk.fr_aldehyde', 'rdk.fr_pyridine',
       'rdk.fr_tetrazole', 'rdk.RingCount', 'rdk.fr_nitro_arom_nonortho',
       'rdk.Chi0v', 'rdk.fr_ArN', 'rdk.NumRotatableBonds', 'c#ClappClass'],
      dtype='object', length=178)


In [4]:
# Seperate test set
train = data.sample(frac=0.8,random_state=0)
test = data.drop(train.index)
#print(len(train))
#print(len(test))
Xtrain, Ytrain = xyPart(train)
Xtest, Ytest = xyPart(test)
print(Xtrain.columns)

Index(['rdk.fr_C_O_noCOO', 'rdk.Chi4v', 'rdk.fr_Ar_COO', 'rdk.fr_SH',
       'rdk.Chi4n', 'rdk.SMR_VSA10', 'rdk.fr_para_hydroxylation',
       'rdk.fr_barbitur', 'rdk.fr_halogen', 'rdk.fr_dihydropyridine',
       ...
       'rdk.MolMR', 'rdk.PEOE_VSA9', 'rdk.fr_aldehyde', 'rdk.fr_pyridine',
       'rdk.fr_tetrazole', 'rdk.RingCount', 'rdk.fr_nitro_arom_nonortho',
       'rdk.Chi0v', 'rdk.fr_ArN', 'rdk.NumRotatableBonds'],
      dtype='object', length=177)


In [5]:
# Normalize
data_stats = Xtrain.describe()
data_stats = data_stats.transpose()
def norm(x):
  return (x - data_stats['mean']) / data_stats['std']
nXtrain = norm(Xtrain)
nXtest = norm(Xtest)
#print(Xtrain.values[0])
#print(nXtrain.values[0])

In [6]:
# Rm columns containing NA
allCol = nXtrain.columns
ncXtrain = nXtrain.dropna(axis='columns')
col = ncXtrain.columns  # After removing NA cols
delCol = []
for c in allCol:
    if c not in col:
        delCol.append(c)
ncXtest = nXtest.drop(delCol, axis=1)
#print(nXtrain.values[0])
#print(ncXtrain.values[0])
#print(ncXtest.columns)

In [7]:
from sklearn.linear_model import LogisticRegression

learner = LogisticRegression()
model = learner.fit(ncXtrain, Ytrain)
print(model)

C:\Users\jgwdk\AppData\Local\Continuum\miniconda3\envs\myAzureEnv\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)


In [8]:
pred = model.predict(ncXtest)

In [9]:
def getCA(pred, Ytest):
    true = false = 0
    for idx in range(len(pred)):
        if Ytest[idx] == pred[idx]:
            true = true + 1
        else:
            false = false + 1
    print(true, false)        
    print("CA ", true/float(true+false))
    CA = true/float(true+false)
    return CA


In [13]:
print("Original model")
getCA(pred, Ytest)

import pickle
fid = open("modelDump.pkl", "wb")
modelFid = pickle.dump(model, fid)
fid.close()
#modelFid = pickle.dumps(model)


Original model
1222 259
CA  0.8251181634031061


In [17]:
fid = open("modelDump.pkl", "rb")
model2 = pickle.load(fid)
fid.close()
#model2 = pickle.loads(modelFid)
pred2 = model2.predict(ncXtest)
print("Loaded model")
getCA(pred2, Ytest)

Loaded model
1222 259
CA  0.8251181634031061


0.8251181634031061